In [1]:
import sys
sys.path.append('../../')
import time
import argparse
import os
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
train_node

,id,amino_acid_seq:String,~label,label,nid
0,b0685,MAKEQTDRTTLDLFAHERRPGRPKTNPLSRDEQLRINKRNQLKRDK...,0,Gene,1668
1,b3582,MRNHQLGIYEKALAKDLSWPERLVLAKSCGFDFVEMSVDETDERLS...,0,Gene,1151
2,b2524,MGLKWTDSREIGEALYDAYPDLDPKTVRFTDMHQWICDLEDFDDDP...,0,Gene,457
3,b1819,MVDTTQTTTEKKLTQSDIRGVFLRSNLFQGSWNFERMQALGFCFSM...,0,Gene,2430
4,b3643,MRPAGRSNNQVRPVTLTRNYTKHAEGSVLVEFGDTKVLCTASIEEG...,0,Gene,1201
...,...,...,...,...,...
2120,b1174,MALLDFFLSRKKNTANIAKERLQIIVAERRRSDAEPHYLPQLRKDI...,0,Gene,2763
2121,b4046,MEKTTTQELLAQAEKICAQRNVRLTPQRLEVLRLMSLQDGAISAYD...,1,Gene,905
2122,b4327,MDDCGAILHNIETKWLYDFLTLEKCRNFSQAAVSRNVSQPAFSRRI...,1,Gene,1096
2123,b3340,MARTTPIARYRNIGISAHIDAGKTTTTERILFYTGVNHKIGEVHDG...,0,Gene,235


In [2]:
train_node = pd.read_csv("../../data/Process/train_node.csv")
test_node = pd.read_csv("../../data/Process/test_node.csv")
test_node

,id,amino_acid_seq:String,~label,label,nid
0,b3064,MRVLGIETSCDETGIAIYDDEKGLLANQLYSQVKLHADYGGVVPEL...,0,Gene,6
1,b3594,MSNTYQKRKASKEYGLYNQCKKLNDDELFRLLDDHNSLKRISSARV...,0,Gene,1171
2,b2918,MINEATLAESIRRLRQGERATLAQAMTLVESRHPRHQALSTQLLDA...,0,Gene,99
3,b4015,MKTRTQQIEELQKEWTQPRWEGITRPYSAEDVVKLRGSVNPECTLA...,0,Gene,927
4,b3176,MSNRKYFGTDGIRGRVGDAPITPDFVLKLGWAAGKVLARHGSRKII...,0,Gene,314
...,...,...,...,...,...
905,b4396,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,1,Gene,968
906,b3943,MSAAGKSNPLAISGLVVLTLIWSYSWIFMKQVTSYIGAFDFTALRC...,0,Gene,877
907,b1204,MTIKLIVGLANPGAEYAATRHNAGAWFVDLLAERLRAPLREEAKFF...,0,Gene,2834
908,b2224,MKNCVIVSAVRTAIGSFNGSLASTSAIDLGATVIKAAIERAKIDSQ...,0,Gene,2545


In [4]:
2125 +910

3035

In [9]:
# Convert DataFrame to FASTA
fasta_lines = []
for index, row in test_node.iterrows():
    header = f">{row['id']}"
    sequence = row['amino_acid_seq:String']
    fasta_lines.append(f"{header}\n{sequence}")

# Write to FASTA file
fasta_filename = "output/test.fasta"
with open(fasta_filename, 'w') as fasta_file:
    fasta_file.write('\n'.join(fasta_lines))

##### DeepTFPredict

In [ ]:
# ! git clone https://bitbucket.org/kaistsystemsbiology/deeptfactor.git
# ! cd deeptfactor
# ! conda env create -f environment.yml
# ! conda activate deeptfactor
# ! python tf_running.py -i output/test.fasta -o output/ -g cuda:0

In [11]:
deeptfpredcit = pd.read_table(f"./output/prediction_result.txt")
deeptfpredcit

,sequence_ID,prediction,score
0,b3064,False,0.0067
1,b3594,False,0.3703
2,b2918,False,0.0139
3,b4015,False,0.0086
4,b3176,False,0.0084
...,...,...,...
897,b4396,True,0.9911
898,b3943,False,0.0057
899,b1204,False,0.0113
900,b2224,False,0.0084


In [12]:
dp_pd = pd.merge(test_node,deeptfpredcit,left_on='id', right_on='sequence_ID')
dp_pd

,id,amino_acid_seq:String,~label,label,nid,sequence_ID,prediction,score
0,b3064,MRVLGIETSCDETGIAIYDDEKGLLANQLYSQVKLHADYGGVVPEL...,0,Gene,6,b3064,False,0.0067
1,b3594,MSNTYQKRKASKEYGLYNQCKKLNDDELFRLLDDHNSLKRISSARV...,0,Gene,1171,b3594,False,0.3703
2,b2918,MINEATLAESIRRLRQGERATLAQAMTLVESRHPRHQALSTQLLDA...,0,Gene,99,b2918,False,0.0139
3,b4015,MKTRTQQIEELQKEWTQPRWEGITRPYSAEDVVKLRGSVNPECTLA...,0,Gene,927,b4015,False,0.0086
4,b3176,MSNRKYFGTDGIRGRVGDAPITPDFVLKLGWAAGKVLARHGSRKII...,0,Gene,314,b3176,False,0.0084
...,...,...,...,...,...,...,...,...
897,b4396,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...,1,Gene,968,b4396,True,0.9911
898,b3943,MSAAGKSNPLAISGLVVLTLIWSYSWIFMKQVTSYIGAFDFTALRC...,0,Gene,877,b3943,False,0.0057
899,b1204,MTIKLIVGLANPGAEYAATRHNAGAWFVDLLAERLRAPLREEAKFF...,0,Gene,2834,b1204,False,0.0113
900,b2224,MKNCVIVSAVRTAIGSFNGSLASTSAIDLGATVIKAAIERAKIDSQ...,0,Gene,2545,b2224,False,0.0084


In [13]:
dp_report = pd.DataFrame(classification_report(dp_pd["~label"].values, dp_pd["prediction"].values, zero_division=0, output_dict=True))
dp_report

,0,1,accuracy,macro avg,weighted avg
precision,0.996381,0.808219,0.981153,0.902300,0.983448
recall,0.983333,0.951613,0.981153,0.967473,0.981153
f1-score,0.989814,0.874074,0.981153,0.931944,0.981859
support,840.000000,62.000000,0.981153,902.000000,902.000000


In [19]:
np.savez("../../output/baseline/DeepTFpredict.npz", label=dp_pd["~label"].values, prob=dp_pd["prediction"].values)